# Sequencing with audio-dsp

This notebook demonstrates algorithmic composition and sequencing:
- Sample-based drum sequencing
- Euclidean rhythms
- Melodic sequencing with scales
- Arpeggiators
- Chord progressions
- Full arrangement

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Metallicode/python_audio_dsp/blob/master/examples/Sequencing.ipynb)

## Installation

Run this cell to install the library (required for Google Colab):

In [ ]:
!pip install audio-dsp -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import os

from audio_dsp.utils import load_audio, save_audio, normalize_audio

SAMPLE_RATE = 44100

# Visualization helpers
def plot_waveform(audio, sr, title="Waveform", figsize=(12, 3), color='#2563eb'):
    """Plot a waveform with time axis."""
    plt.figure(figsize=figsize)
    time = np.arange(len(audio)) / sr
    plt.plot(time, audio, color=color, linewidth=0.5)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title(title)
    plt.xlim(0, time[-1])
    plt.ylim(-1.1, 1.1)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_pattern(patterns, steps=16, title="Drum Pattern"):
    """Visualize a drum pattern as a grid."""
    instruments = list(patterns.keys())
    fig, ax = plt.subplots(figsize=(12, len(instruments) * 0.6 + 0.5))
    
    colors = {'kick': '#dc2626', 'snare': '#2563eb', 'hihat': '#16a34a', 'clap': '#9333ea', 'ride': '#eab308'}
    
    for i, (name, step_list) in enumerate(patterns.items()):
        y = len(instruments) - 1 - i
        color = colors.get(name, '#6b7280')
        for step in step_list:
            ax.add_patch(plt.Rectangle((step, y), 0.8, 0.8, facecolor=color, edgecolor='white', linewidth=2))
        ax.text(-0.5, y + 0.4, name.capitalize(), ha='right', va='center', fontsize=10, fontweight='bold')
    
    ax.set_xlim(-2, steps)
    ax.set_ylim(-0.2, len(instruments))
    ax.set_xticks(range(steps))
    ax.set_xticklabels([str(i+1) for i in range(steps)])
    ax.set_yticks([])
    ax.set_xlabel('Step')
    ax.set_title(title)
    ax.set_aspect('equal')
    
    # Add beat markers
    for beat in range(0, steps, 4):
        ax.axvline(beat - 0.1, color='gray', linestyle='--', alpha=0.3, linewidth=1)
    
    plt.tight_layout()
    plt.show()

## For Google Colab Users

If running on Colab, you'll need to upload sample files or use synthesized drums (see alternative cell below).

## 1. Loading One-Shot Samples

First, let's load the drum samples from the assets folder.

In [ ]:
# Option 1: Load drum one-shots from files
# Uncomment and run this cell if you have the sample files
samples_dir = "assets/one-shots"

try:
    kick, sr = load_audio(os.path.join(samples_dir, "RX11_BD.wav"))
    snare, _ = load_audio(os.path.join(samples_dir, "RX11_SNR.wav"))
    clap, _ = load_audio(os.path.join(samples_dir, "RX11_CLAP.wav"))
    hihat, _ = load_audio(os.path.join(samples_dir, "RX11_CLHAT.wav"))
    ride, _ = load_audio(os.path.join(samples_dir, "RX11_RIDE.wav"))
    
    print("Loaded samples from files:")
    for name, sample in [("Kick", kick), ("Snare", snare), ("Clap", clap), ("Hi-hat", hihat), ("Ride", ride)]:
        print(f"  {name}: {len(sample)} samples ({len(sample)/sr*1000:.0f}ms)")
        
except FileNotFoundError:
    print("Sample files not found. Run the next cell to use synthesized drums instead.")

In [ ]:
# Option 2: Create synthesized drums (works on Colab without uploading files)
# Run this cell if you don't have sample files

def create_kick(sr=44100, duration=0.3):
    """Synthesize a kick drum sound."""
    t = np.linspace(0, duration, int(sr * duration), endpoint=False)
    # Pitch envelope: starts high, drops fast
    freq = 150 * np.exp(-30 * t) + 40
    phase = np.cumsum(freq) * 2 * np.pi / sr
    kick = np.sin(phase) * np.exp(-8 * t)
    return kick.astype(np.float32)

def create_snare(sr=44100, duration=0.2):
    """Synthesize a snare drum sound."""
    t = np.linspace(0, duration, int(sr * duration), endpoint=False)
    # Body: sine at 180 Hz
    body = np.sin(2 * np.pi * 180 * t) * np.exp(-15 * t)
    # Noise for the snare wires
    noise = np.random.uniform(-1, 1, len(t)) * np.exp(-10 * t)
    snare = body * 0.6 + noise * 0.4
    return (snare / np.max(np.abs(snare)) * 0.9).astype(np.float32)

def create_hihat(sr=44100, duration=0.08):
    """Synthesize a hi-hat sound."""
    t = np.linspace(0, duration, int(sr * duration), endpoint=False)
    # Filtered noise
    noise = np.random.uniform(-1, 1, len(t))
    # High-pass effect via differentiation
    hihat = np.diff(np.append([0], noise)) * np.exp(-40 * t)
    return (hihat / np.max(np.abs(hihat)) * 0.6).astype(np.float32)

def create_clap(sr=44100, duration=0.15):
    """Synthesize a clap sound."""
    t = np.linspace(0, duration, int(sr * duration), endpoint=False)
    # Multiple bursts of noise
    clap = np.zeros_like(t)
    for i in range(4):
        offset = int(i * sr * 0.01)
        if offset < len(t):
            burst_len = min(int(sr * 0.01), len(t) - offset)
            clap[offset:offset + burst_len] += np.random.uniform(-1, 1, burst_len) * (0.5 ** i)
    clap *= np.exp(-20 * t)
    return (clap / np.max(np.abs(clap)) * 0.8).astype(np.float32)

# Create synthesized drum samples
kick = create_kick(SAMPLE_RATE)
snare = create_snare(SAMPLE_RATE)
hihat = create_hihat(SAMPLE_RATE)
clap = create_clap(SAMPLE_RATE)
ride = create_hihat(SAMPLE_RATE, duration=0.2)  # Longer hihat for ride

print("Created synthesized drum samples:")
for name, sample in [("Kick", kick), ("Snare", snare), ("Clap", clap), ("Hi-hat", hihat), ("Ride", ride)]:
    print(f"  {name}: {len(sample)} samples ({len(sample)/SAMPLE_RATE*1000:.0f}ms)")
    display(Audio(sample, rate=SAMPLE_RATE))

## 2. Basic Step Sequencer

Create drum patterns using a grid-based approach.

In [ ]:
def create_pattern(samples, patterns, bpm=120, steps=16):
    """
    Create a drum pattern from samples and step patterns.
    
    Args:
        samples: dict of {name: audio_array}
        patterns: dict of {name: list of step indices}
        bpm: tempo in beats per minute
        steps: number of steps (16 = one bar of 16th notes)
    """
    # Calculate step length in samples
    beat_duration = 60.0 / bpm
    step_duration = beat_duration / 4  # 16th notes
    step_samples = int(step_duration * SAMPLE_RATE)
    total_samples = step_samples * steps
    
    # Create output buffer
    output = np.zeros(total_samples)
    
    # Place samples at step positions
    for name, step_list in patterns.items():
        sample = samples[name]
        for step in step_list:
            start = step * step_samples
            end = min(start + len(sample), total_samples)
            sample_len = end - start
            output[start:end] += sample[:sample_len]
    
    return normalize_audio(output, peak=0.9)

# Define samples dict
drum_samples = {
    "kick": kick,
    "snare": snare,
    "hihat": hihat,
    "clap": clap
}

In [ ]:
# Basic 4-on-the-floor pattern
# Steps: 0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15
#        1  .  .  .  2  .  .  .  3  .  .  .  4  .  .  .

four_on_floor = {
    "kick":  [0, 4, 8, 12],           # Kick on every beat
    "snare": [4, 12],                  # Snare on 2 and 4
    "hihat": [0, 2, 4, 6, 8, 10, 12, 14]  # Hats on 8th notes
}

# Visualize the pattern grid
plot_pattern(four_on_floor, title="4-on-the-floor Pattern")

pattern1 = create_pattern(drum_samples, four_on_floor, bpm=120)

# Show waveform
plot_waveform(pattern1, SAMPLE_RATE, "4-on-the-floor (120 BPM)")
Audio(pattern1, rate=SAMPLE_RATE)

In [ ]:
# Breakbeat-style pattern
breakbeat = {
    "kick":  [0, 6, 10],
    "snare": [4, 12],
    "hihat": [0, 2, 4, 6, 8, 10, 12, 14],
    "clap":  [4, 12]
}

# Visualize the pattern grid
plot_pattern(breakbeat, title="Breakbeat Pattern")

pattern2 = create_pattern(drum_samples, breakbeat, bpm=140)

# Show waveform
plot_waveform(pattern2, SAMPLE_RATE, "Breakbeat pattern (140 BPM)")
Audio(pattern2, rate=SAMPLE_RATE)

In [ ]:
# Loop the pattern 4 times
looped = np.tile(pattern2, 4)

# Show waveform of looped pattern
plot_waveform(looped, SAMPLE_RATE, "Breakbeat (4 bars)")
Audio(looped, rate=SAMPLE_RATE)

## 3. Euclidean Rhythms

Generate mathematically distributed rhythms using the Euclidean algorithm.

In [ ]:
def euclidean_rhythm(hits, steps, rotation=0):
    """
    Generate a Euclidean rhythm pattern.
    
    Args:
        hits: number of hits to distribute
        steps: total number of steps
        rotation: rotate pattern by this many steps
    
    Returns:
        List of step indices where hits occur
    """
    if hits >= steps:
        return list(range(steps))
    if hits == 0:
        return []
    
    pattern = []
    bucket = 0
    
    for i in range(steps):
        bucket += hits
        if bucket >= steps:
            bucket -= steps
            pattern.append((i + rotation) % steps)
    
    return sorted(pattern)

# Test some classic Euclidean rhythms
print("E(3,8) - Cuban tresillo:", euclidean_rhythm(3, 8))
print("E(5,8) - Cinquillo:", euclidean_rhythm(5, 8))
print("E(7,16) - West African bell:", euclidean_rhythm(7, 16))

In [ ]:
# Create a polyrhythmic pattern using Euclidean rhythms
euclidean_pattern = {
    "kick":  euclidean_rhythm(4, 16),          # E(4,16)
    "snare": euclidean_rhythm(3, 16, rotation=4),  # E(3,16) rotated
    "hihat": euclidean_rhythm(7, 16),          # E(7,16)
    "clap":  euclidean_rhythm(2, 16, rotation=2)   # E(2,16) rotated
}

print("Euclidean pattern step indices:")
for name, steps in euclidean_pattern.items():
    print(f"  {name}: {steps}")

# Visualize the pattern grid
plot_pattern(euclidean_pattern, title="Euclidean Polyrhythm Pattern")

eucl_beat = create_pattern(drum_samples, euclidean_pattern, bpm=110)
eucl_loop = np.tile(eucl_beat, 4)

# Show waveform
plot_waveform(eucl_loop, SAMPLE_RATE, "Euclidean polyrhythm (4 bars, 110 BPM)")
Audio(eucl_loop, rate=SAMPLE_RATE)

## 4. Melodic Sequencing with Scales

Generate melodies using scales and the built-in synths.

In [ ]:
from audio_dsp.utils import generate_scale
from audio_dsp.synth import SubtractiveSynth

# Generate a minor pentatonic scale
root_freq = 220  # A3
scale_intervals = [0, 3, 5, 7, 10, 12]  # Minor pentatonic

scale_freqs = [root_freq * (2 ** (i/12)) for i in scale_intervals]
print("A minor pentatonic frequencies:", [f"{f:.1f} Hz" for f in scale_freqs])

# Create synth
synth = SubtractiveSynth(sample_rate=SAMPLE_RATE)
synth.osc_wave = "saw"
synth.filter_cutoff = 1500
synth.filter_resonance = 2.0
synth.attack = 0.01
synth.decay = 0.1
synth.sustain = 0.5
synth.release = 0.2

In [ ]:
# Random melody generator
np.random.seed(42)

def generate_melody(scale_freqs, n_notes=16, note_duration=0.15, synth=None):
    """Generate a random melody from a scale."""
    melody = []
    
    for _ in range(n_notes):
        # Pick a random note from the scale
        freq = np.random.choice(scale_freqs)
        # Occasionally add octave variation
        if np.random.random() > 0.7:
            freq *= np.random.choice([0.5, 2.0])
        
        note = synth.synthesize(freq, note_duration)
        melody.append(note)
    
    return np.concatenate(melody)

melody = generate_melody(scale_freqs, n_notes=32, note_duration=0.12, synth=synth)

# Show waveform
plot_waveform(melody, SAMPLE_RATE, "Random Pentatonic Melody")
Audio(melody, rate=SAMPLE_RATE)

In [ ]:
# Combine melody with drums
# Pad melody to match drum loop length
drum_loop = np.tile(pattern2, 4)

# Repeat melody to match length
melody_repeated = np.tile(melody, int(np.ceil(len(drum_loop) / len(melody))))[:len(drum_loop)]

# Mix
combined = drum_loop * 0.7 + melody_repeated * 0.5
combined = normalize_audio(combined, peak=0.9)

# Show waveform
plot_waveform(combined, SAMPLE_RATE, "Drums + Melody Combined")
Audio(combined, rate=SAMPLE_RATE)

## 5. Arpeggiator

Create arpeggiated patterns from chords.

In [ ]:
def arpeggiator(chord_freqs, pattern="up", octaves=2, note_duration=0.1, synth=None):
    """
    Create an arpeggiated pattern from chord frequencies.
    
    Args:
        chord_freqs: list of frequencies in the chord
        pattern: "up", "down", "updown", "random"
        octaves: number of octaves to span
        note_duration: duration of each note
        synth: synthesizer to use
    """
    # Extend chord across octaves
    all_freqs = []
    for oct in range(octaves):
        for freq in chord_freqs:
            all_freqs.append(freq * (2 ** oct))
    
    # Create pattern order
    if pattern == "up":
        order = list(range(len(all_freqs)))
    elif pattern == "down":
        order = list(range(len(all_freqs) - 1, -1, -1))
    elif pattern == "updown":
        order = list(range(len(all_freqs))) + list(range(len(all_freqs) - 2, 0, -1))
    elif pattern == "random":
        order = list(range(len(all_freqs)))
        np.random.shuffle(order)
    
    # Generate notes
    notes = []
    for i in order:
        note = synth.synthesize(all_freqs[i], note_duration)
        notes.append(note)
    
    return np.concatenate(notes)

# A minor chord: A, C, E
am_chord = [220, 261.6, 329.6]

# Create different arpeggio patterns
arp_synth = SubtractiveSynth(sample_rate=SAMPLE_RATE)
arp_synth.osc_wave = "square"
arp_synth.filter_cutoff = 2000
arp_synth.attack = 0.005
arp_synth.decay = 0.05
arp_synth.sustain = 0.3
arp_synth.release = 0.1

# Arpeggio UP
arp_up = arpeggiator(am_chord, "up", octaves=2, note_duration=0.08, synth=arp_synth)
arp_up_looped = np.tile(arp_up, 4)
print("Arpeggio UP:")
plot_waveform(arp_up_looped, SAMPLE_RATE, "Arpeggio UP (Am chord)")
display(Audio(arp_up_looped, rate=SAMPLE_RATE))

# Arpeggio UP-DOWN
arp_updown = arpeggiator(am_chord, "updown", octaves=2, note_duration=0.08, synth=arp_synth)
arp_updown_looped = np.tile(arp_updown, 3)
print("\nArpeggio UP-DOWN:")
plot_waveform(arp_updown_looped, SAMPLE_RATE, "Arpeggio UP-DOWN (Am chord)")
display(Audio(arp_updown_looped, rate=SAMPLE_RATE))

## 6. Chord Progression Sequencer

Play through a chord progression with arpeggios.

In [ ]:
# Define chord progression: Am - F - C - G
chords = {
    "Am": [220.0, 261.6, 329.6],      # A, C, E
    "F":  [174.6, 220.0, 261.6],      # F, A, C
    "C":  [261.6, 329.6, 392.0],      # C, E, G
    "G":  [196.0, 246.9, 293.7],      # G, B, D
}

progression = ["Am", "F", "C", "G"]

# Generate arpeggiated progression
full_sequence = []
for chord_name in progression:
    chord_freqs = chords[chord_name]
    arp = arpeggiator(chord_freqs, "updown", octaves=2, note_duration=0.1, synth=arp_synth)
    # Repeat arpeggio to fill one bar
    bar_length = int(SAMPLE_RATE * 2)  # 2 seconds per chord
    arp_repeated = np.tile(arp, int(np.ceil(bar_length / len(arp))))[:bar_length]
    full_sequence.append(arp_repeated)

chord_sequence = np.concatenate(full_sequence)
chord_sequence = normalize_audio(chord_sequence, peak=0.8)

# Show waveform
plot_waveform(chord_sequence, SAMPLE_RATE, "Chord Progression: Am - F - C - G")
Audio(chord_sequence, rate=SAMPLE_RATE)

## 7. Full Arrangement

Combine all elements into a complete musical piece.

In [ ]:
# Create a 16-bar arrangement
bar_length = int(SAMPLE_RATE * 2)  # 2 seconds per bar at 120 BPM

# Create drum patterns for different sections
intro_drums = create_pattern(drum_samples, {
    "hihat": [0, 4, 8, 12]
}, bpm=120)

verse_drums = create_pattern(drum_samples, four_on_floor, bpm=120)

# Pad patterns to bar length
def pad_to_length(audio, length):
    if len(audio) >= length:
        return audio[:length]
    repeats = int(np.ceil(length / len(audio)))
    return np.tile(audio, repeats)[:length]

# Arrangement structure: 4 bars intro, 8 bars verse, 4 bars outro
arrangement = []

# Intro: just hihats + arpeggio
for i in range(4):
    drums = pad_to_length(intro_drums, bar_length)
    arp = pad_to_length(chord_sequence, bar_length * 4)[i * bar_length:(i+1) * bar_length]
    bar = drums * 0.5 + arp * 0.6
    arrangement.append(bar)

# Verse: full drums + arpeggio
for i in range(8):
    drums = pad_to_length(verse_drums, bar_length)
    chord_idx = i % 4
    arp = pad_to_length(chord_sequence, bar_length * 4)[chord_idx * bar_length:(chord_idx+1) * bar_length]
    bar = drums * 0.7 + arp * 0.5
    arrangement.append(bar)

# Outro: fade out
for i in range(4):
    drums = pad_to_length(intro_drums, bar_length)
    arp = pad_to_length(chord_sequence, bar_length * 4)[i * bar_length:(i+1) * bar_length]
    bar = drums * 0.5 + arp * 0.6
    # Apply fade
    fade = np.linspace(1, 0, bar_length) ** 0.5 if i == 3 else np.ones(bar_length)
    bar = bar * fade
    arrangement.append(bar)

# Combine and normalize
full_track = np.concatenate(arrangement)
full_track = normalize_audio(full_track, peak=0.9)

print(f"Full arrangement: {len(full_track)/SAMPLE_RATE:.1f} seconds")
print("Structure: 4 bars intro -> 8 bars verse -> 4 bars outro")

# Show waveform with arrangement sections
fig, ax = plt.subplots(figsize=(14, 4))
time = np.arange(len(full_track)) / SAMPLE_RATE
ax.plot(time, full_track, color='#2563eb', linewidth=0.3)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title('Full Arrangement')
ax.set_xlim(0, time[-1])
ax.set_ylim(-1.1, 1.1)
ax.grid(True, alpha=0.3)

# Add section markers
sections = [
    (0, 8, "Intro", '#16a34a'),
    (8, 24, "Verse", '#dc2626'),
    (24, 32, "Outro", '#9333ea')
]
for start, end, label, color in sections:
    ax.axvspan(start, end, alpha=0.1, color=color)
    ax.text((start + end) / 2, 0.95, label, ha='center', va='top', fontsize=10, fontweight='bold', color=color)

plt.tight_layout()
plt.show()

Audio(full_track, rate=SAMPLE_RATE)

In [ ]:
# Save the final arrangement
save_audio("output_arrangement.wav", full_track, SAMPLE_RATE)
print("Saved to output_arrangement.wav")

## Next Steps

Explore the other example notebooks:
- **Quickstart.ipynb** - Quick introduction to the library
- **Synthesis.ipynb** - Deep dive into synthesizers
- **Effects.ipynb** - Complete effects reference